<a id='report-storage'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Report Storage and Identity </span> </p>

The Report Identity is a record collecting all information needed to define the granularity in which the calculation are performed.

The Report Storage collects the required data and parameters used in the calculations.

Both Storage and Identity are defined to support the calculations defined in the corresponding [Report Scopes](./ReportScopes).

In [0]:
#!import "ReportConfigurationAndUtils"

# Report Identity

In [0]:
[IdentityAggregationBehaviour(IdentityAggregationBehaviour.Aggregate)]
public record ReportIdentity {
    
    [Dimension(typeof(int), nameof(Year))]
    public int Year { get; init; }

    [Dimension(typeof(int), nameof(Month))]
    public int Month { get; init; }

    [Dimension(typeof(ReportingNode))]
    public string ReportingNode { get; init; }
    
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }

    [Dimension(typeof(Currency), nameof(ContractualCurrency))]
    public string ContractualCurrency { get; init; }
    
    [Dimension(typeof(Currency), nameof(FunctionalCurrency))]
    public string FunctionalCurrency { get; init; }

    [NotAggregated]
    [Dimension(typeof(int), nameof(Projection))]
    public int Projection { get; init; }

    [Dimension(typeof(LiabilityType))]
    public string LiabilityType { get; init; }
    
    [Dimension(typeof(ValuationApproach))]
    public string ValuationApproach { get; init; }
    
    public bool IsReinsurance { get; init; } //TODO use ReinsuranceType
    
    public bool IsOci { get; init; } 
}

# Report Storage

In [0]:
public class ReportStorage {
    protected readonly IWorkspace workspace;
    private readonly Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchicalDimensionCache;
    private readonly Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory reportFactory;
    
    // Current Storage Settings
    public ((int Year, int Month) Period, string ReportingNode, string Scenario, CurrencyType CurrencyType) Args {get; private set;}
    
    // Cache for Aux Data
    private Dictionary<(int year, int month), Dictionary<string, Dictionary<FxPeriod, double>>> exchangeRatesByCurrencyByFxTypeAndPeriod = new(); // Fx Rates
    private Dictionary<(int year, int month), Dictionary<AocStep, FxPeriod>> fxPeriodsByAocStepAndPeriod = new(); // FxParameter
    
    public HashSet<string> EstimateTypesWithoutAoc {get; private set;}
    
    // Cache for Variables and Parameters
    private Dictionary<((int year, int month) period, string reportingNode, string scenario), Dictionary<ReportIdentity, Dictionary<string, IDataCube<ReportVariable>>>> variablesDictionary = new();
        
    // Constructor
    public ReportStorage(IWorkspace workspace, Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory reportFactory) {
        this.workspace = workspace;
        this.hierarchicalDimensionCache = workspace.ToHierarchicalDimensionCache();
        this.reportFactory = reportFactory;
    }
    
    // Initializers
    public async Task InitializeReportIndependentCacheAsync() {
        // Hierarchical Dimensions
        await hierarchicalDimensionCache.InitializeAsync<LineOfBusiness>();
        await hierarchicalDimensionCache.InitializeAsync<AmountType>();  
        await hierarchicalDimensionCache.InitializeAsync<VariableType>();
        await hierarchicalDimensionCache.InitializeAsync<ReportingNode>();
    }
    
    public async Task InitializeAsync((int year, int month) period, string reportingNode, string scenario, CurrencyType currencyType) {
        // Report Dependent Cache

        // Setting the Args --> Temp for the moment
        Args = (period, reportingNode, scenario, currencyType);
        
        EstimateTypesWithoutAoc = (await workspace.Query<EstimateType>().Where(x => x.StructureType != StructureType.AoC).Select(x => x.SystemName).ToArrayAsync()).ToHashSet();
        
        // FX && Fx Parameters
        if(!exchangeRatesByCurrencyByFxTypeAndPeriod.TryGetValue(period, out var exchangeRatesByCurrencyByFxType) || !fxPeriodsByAocStepAndPeriod.TryGetValue(period, out var fxPeriodsByAocStep)) {
            exchangeRatesByCurrencyByFxType = await workspace.GetExchangeRatesDictionaryAsync(period.year, period.month);
            exchangeRatesByCurrencyByFxTypeAndPeriod.Add(period, exchangeRatesByCurrencyByFxType);
            fxPeriodsByAocStep = (await workspace.LoadAocStepConfigurationAsync(period.year, period.month)).Where(x => x.FxPeriod != FxPeriod.NotApplicable).ToDictionary(x => new AocStep(x.AocType, x.Novelty), x => (FxPeriod)x.FxPeriod);
            fxPeriodsByAocStepAndPeriod.Add(period, fxPeriodsByAocStep);
        }
        
        // Variables
        foreach(var rn in GetLeaves<ReportingNode>(reportingNode)) {
            if(!variablesDictionary.TryGetValue((period, rn, scenario), out var variablesByIdentity)) {
                variablesByIdentity = (await workspace.QueryReportVariablesAsync((period.year, period.month, rn, scenario)))
                    .ToDictionaryGrouped(x => new ReportIdentity {
                        Year = period.year,
                        Month = period.month,
                        ReportingNode = x.ReportingNode,
                        Scenario = x.Scenario,
                        Projection = x.Projection,
                        ContractualCurrency = x.ContractualCurrency,
                        FunctionalCurrency = x.FunctionalCurrency,
                        ValuationApproach = x.ValuationApproach,
                        LiabilityType = x.LiabilityType,
                        IsReinsurance = x.IsReinsurance,
                        IsOci = !string.IsNullOrWhiteSpace(x.OciType) },
                                         x => x.ToDictionaryGrouped(y => y.EstimateType,
                                                                    y => y.ToArray().ToDataCube()));
            
                variablesDictionary.Add((period, rn, scenario), variablesByIdentity);
            }
        }
    }
    
    // Getters for Data
    public IDataCube<ReportVariable> GetVariables(ReportIdentity reportIdentity, params string[] estimateTypes)
        => (!variablesDictionary.TryGetValue(((reportIdentity.Year, reportIdentity.Month), reportIdentity.ReportingNode, reportIdentity.Scenario), out var variablesByIdentity) || !variablesByIdentity.TryGetValue(reportIdentity, out var variablesByEstimateType))
        ? Enumerable.Empty<ReportVariable>().ToDataCube()
        : estimateTypes.Length switch {
                0 => variablesByEstimateType.SelectMany(x => x.Value).ToDataCube(),
                1 => variablesByEstimateType.TryGetValue(estimateTypes.First(), out var variables)
                    ? variables.ToDataCube()
                    : Enumerable.Empty<ReportVariable>().ToDataCube(),
                _ => estimateTypes.Select(et => variablesByEstimateType.TryGetValue(et, out var variables)
                                          ? variables.ToDataCube()
                                          : Enumerable.Empty<ReportVariable>())
                    .Aggregate((x, y) => x.Concat(y))
                    .ToDataCube()
        };
    
    // Other getters
    public IWorkspace Workspace => workspace;
    public Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory Report => reportFactory;

    public Systemorph.Vertex.Hierarchies.IHierarchy<T> GetHierarchy<T>() where T : class, IHierarchicalDimension => hierarchicalDimensionCache.Get<T>();
    
    public HashSet<(ReportIdentity, CurrencyType)> GetIdentities((int year, int month) period, string reportingNode, string scenario, CurrencyType currencyType)
        => GetLeaves<ReportingNode>(reportingNode)
        .SelectMany(rn => variablesDictionary.TryGetValue((period, rn, scenario), out var inner) ? inner.Keys.Select(x => (x, currencyType)) : Enumerable.Empty<(ReportIdentity, CurrencyType)>())
        .ToHashSet();
    
    public double GetFx((int year, int month) period, string currentCurrency, string targetCurrency, FxPeriod fxPeriod) {
        if (currentCurrency == targetCurrency) return 1;
        if(!exchangeRatesByCurrencyByFxTypeAndPeriod.TryGetValue(period, out var exchangeRatesByCurrencyByFxType))
            throw new Exception ($"No exchange rates for Period {period} were found.");
         return GetCurrencyToGroupFx(exchangeRatesByCurrencyByFxType, currentCurrency, fxPeriod, GroupCurrency)
             / GetCurrencyToGroupFx(exchangeRatesByCurrencyByFxType, targetCurrency, fxPeriod, GroupCurrency);
    }
    
    public FxPeriod GetFxPeriod((int year, int month) period, string aocType, string novelty) => fxPeriodsByAocStepAndPeriod[period][new AocStep(aocType, novelty)];
      
    // Helpers
    public HashSet<string> GetLeaves<T>(string systemName) where T : class, IHierarchicalDimension {
        var descendants = hierarchicalDimensionCache.Get<T>(systemName).Descendants(includeSelf: true);
        return descendants.Where(x => !descendants.Select(y => y.Parent).Contains(x.SystemName)).Select(x => x.SystemName).ToHashSet();
    }
}